In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import requests
import os
import csv

In [1]:
# 이미지를 저장할 디렉터리 확인 및 생성
image_dir = './images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
data = []  # 데이터를 저장할 리스트
# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.headless = True  # 브라우저 창을 띄우지 않는 헤드리스 모드
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 웹 페이지 로드
url = "https://pcmap.place.naver.com/restaurant/1235520676/menu/list"
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기
time.sleep(1)  # 로딩 시간에 따라 조정이 필요할 수 있음

# 이미지가 포함된 div 태그를 찾음
img_tags = driver.find_elements(By.CLASS_NAME, 'K0PDV')

# 이미지 URL 및 음식 이름 추출
for index, img_tag in enumerate(img_tags):
    style_attr = img_tag.get_attribute('style')
    img_url_match = re.search(r"background-image: url\(\"(.*?)\"\)", style_attr)
    
    if img_url_match:
        img_url = img_url_match.group(1)
        print("찾은 이미지 URL:", img_url)

        # Find the corresponding food name for the current image
        food_name_element = img_tag.find_element(By.CLASS_NAME, 'place_blind')
        food_name = food_name_element.text.strip()
        print("찾은 음식 이름:", food_name)

        # 이미지 다운로드
        img_response = requests.get(img_url)
        img_response.raise_for_status()
        data.append({'FoodName': food_name, 'ImageURL': img_url})

        # 이미지 파일 저장
        img_extension = img_url.split('.')[-1].split('?')[0]  # 쿼리 파라미터 제거
        img_filename = f'{food_name}_{index}.{img_extension}'
        img_filepath = os.path.join(image_dir, img_filename)
        with open(img_filepath, 'wb') as img_file:
            img_file.write(img_response.content)
        print(f"이미지 저장됨: {img_filepath}")
    else:
        print("이미지 URL을 찾을 수 없습니다.")

# 드라이버 종료
driver.quit()

csv_file_path = 'Food_data.csv'
csv_columns = ['FoodName', 'ImageURL']

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print("다운로드 및 CSV 파일 생성 완료.")


찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230526_64%2F1685109309274AJhif_JPEG%2FF9F67227-5BB2-4C98-8AC4-AB6E2E79D2FA.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_0.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230603_288%2F1685804019331S56xK_JPEG%2F9B5B4481-EFEA-4328-B896-D04F5282689A.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_1.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230508_58%2F1683532471315q8iTU_JPEG%2FE44AED3A-D00B-43D4-837A-A267DF6E10E1.jpeg
찾은 음식 이름: 
이미지 저장됨: ./images\_2.jpeg
찾은 이미지 URL: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230429_66%2F1682697744665pHPt3_JPEG%2F7D0A1705-BBD0-49D0-B5CD-54348939328F.jpeg
찾은 음식 이름: 업체
이미지 저장됨: ./images\업체_3.jpeg
찾은 이미지 URL: https://search.

In [2]:
url = "https://g.tivan.naver.com/gfa/JdW02o_YKZa0kg7KjNOe4AgZDIazaLayKHRGuIclEmtusgGrb0_ZFXrmBWgScgr4m2MHccLCKGU9bYDMLHrWMvM51GDSjcVzM8lB5n74XrTjzd898VFNLDsYY1OS3n40xFFh4yqr5xniCeYF69ZwREqoIwFLnFQTNcCyw5timdz2muqtMr2OM2x384-DkTlbfxUvJmgpvSjFaN52wWn4Z4AlyAXCFqWByqhjCnoZ2IokW4TZlg27vxg4DMVWUgMArE44AH1Eb5-pGvxulEap03iWaBuXnIVx41jwugTbPYJPt3QjcMUT-ny3ies0QVA6zgIpNDkeCqTb06GVXnwmsFLz5yXWY9_MNdjb7pGfi3WbxxrhGkx-i6UzdFOZ8qyzEu4pQTi1FTfXbuk_dZwrl55bMa9OOr308KTv5oaNaM5OowmiQjD5bMKYosY5iPwhWJfryjYlFJgEq2JU_yaWqa0MjX-idMhC8jizxfNC52Z6cpdhPMoI2p03v20z-ZspWHGrwaWnVjzyKwfGkcgzODhFV3vDGpMY5ApYvxpBeD6NwDTFYgG0FBYWip1on0JRpwzPNWdPVEkYCJG1Gej3-8o9E9UqQ_wa9ui7xAseUXwgVhAUj93AA0r6zK-2tRh9ZDnmeXPTSQRupjKJx4ZPON0dIIZ5yGzaLleoQ0_hjQ41rxiT5t05erVSujTfKto8243SKzjVQOXx6GloTP2L_zLry0ElMFGSb-aqX_B0OJ1m8HxOtQ6cdWSE0krjnqp1JTp3qz6IhWg87qhbUaT1jge3kHvOPkazMArFr_bm6Azejwj-9PJyhvI86GORbNhkVoVkYffVA_vUE9DuTguwwNquUfUsJGFN4NXZanofU-VUHS4vVZDMxC8A1LKl1WZ2SGKVP0LOMI9AcxGy-wqpbqG3yDKajvacwsjC42atsQZ34EzfHkdeKkX6ldlVbPkA-4w4BTwe8UWwFKxsL0Qyn1xmFjLqhPg7IU9L4i_BKTrqOOKbwHrrR_98Kz6o8PDc8aeri1Usmi5KBwDIlWZSD1Q3uHH8QXZS9vKat78u_1qCxbPHkVinV2S9FldycBWGR44ybrzWJ6hXRJauboSkoHEmPVm3xBEfT8F-IeXLNABZojWrxkc1PnqL5tWNp4LxmxsuHmGoEdcZSWFOp8CloEBGx3V2bWqM8jyO8fjImU0=/?dummy=1707292535290"

In [6]:
head = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Host": "g.tivan.naver.com",
    "Origin": "https://pcmap.place.naver.com",
    "Referer": "https://pcmap.place.naver.com/restaurant/1235520676/menu/list",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"
}

In [7]:
naver_r = requests.get(url,headers=head)

In [8]:
naver_r.text

''

In [9]:
url_l = "https://pcmap.place.naver.com/restaurant/1235520676/review/visitor?reviewMenu=%EB%B0%B1%EA%B9%80%EC%B9%98&reviewSort=recent#"

In [10]:
payload = {
    "0": {
        "operationName": "getVisitorReviews",
        "variables": {
            "id": "1235520676",
            "input": {
                "businessId": "1235520676",
                "businessType": "restaurant",
                "item": "0",
                "bookingBusinessId": "920095",
                "page": 3,
                "size": 10,
                "sort": "recent",
                "getReactions": True,
                "getTrailer": True,
                "getUserStats": True,
                "includeContent": True,
                "includeReceiptPhotos": True,
                "isPhotoUsed": False,
                "menu": "백김치",
                "cidList": ["220036", "220039", "220091", "220861"]
            }
        },
        "query": "query getVisitorReviews($input: VisitorReviewsInput) {visitorReviews(input: $input) {items {id rating author {id nickname from imageUrl borderImageUrl objectId url review {totalCount imageCount avgRating __typename} theme {totalCount __typename} isFollowing followerCount followRequested __typename} body thumbnail media {type thumbnail thumbnailRatio class videoId videoOriginSource trailerUrl __typename} tags status visitCount viewCount visited created reply {editUrl body editedBy created date replyTitle isReported isSuspended __typename} originType item {name code options __typename} language highlightOffsets apolloCacheId translatedText businessName showBookingItemName bookingItemName votedKeywords {code iconUrl iconCode displayName __typename} userIdno loginIdno receiptInfoUrl reactionStat {id typeCount {name count __typename} totalCount __typename} hasViewerReacted {id reacted __typename} nickname showPaymentInfo __typename} starDistribution {score count __typename} hideProductSelectBox total showRecommendationSort itemReviewStats {score count itemId starDistribution {score count __typename} __typename} reactionTypes {name emojiUrl label __typename} __typename}}",
    }
}


In [13]:
r = requests.post(url_l, json=payload)

In [14]:
r.text

'Method Not Allowed'

In [14]:
import requests

url_l = "https://pcmap-api.place.naver.com/graphql"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://pcmap.place.naver.com/restaurant/1235520676/review/visitor?reviewMenu=%EB%B0%B1%EA%B9%80%EC%B9%98&reviewSort=recent",
    "Origin": "https://pcmap.place.naver.com"
}

payload = {
    "0": {
        "operationName": "getVisitorReviews",
        "variables": {
            "id": "1235520676",
            "input": {
                "businessId": "1235520676",
                "businessType": "restaurant",
                "item": "0",
                "bookingBusinessId": "920095",
                "page": 5,
                "size": 10,
                "sort": "recent",
                "getReactions": True,
                "getTrailer": True,
                "getUserStats": True,
                "includeContent": True,
                "includeReceiptPhotos": True,
                "isPhotoUsed": False,
                "menu": "백김치",
                "cidList": ["220036", "220039", "220091", "220861"]
            }
        },
        "query": "query getVisitorReviews($input: VisitorReviewsInput) {visitorReviews(input: $input) {items {id rating author {id nickname from imageUrl borderImageUrl objectId url review {totalCount imageCount avgRating __typename} theme {totalCount __typename} isFollowing followerCount followRequested __typename} body thumbnail media {type thumbnail thumbnailRatio class videoId videoOriginSource trailerUrl __typename} tags status visitCount viewCount visited created reply {editUrl body editedBy created date replyTitle isReported isSuspended __typename} originType item {name code options __typename} language highlightOffsets apolloCacheId translatedText businessName showBookingItemName bookingItemName votedKeywords {code iconUrl iconCode displayName __typename} userIdno loginIdno receiptInfoUrl reactionStat {id typeCount {name count __typename} totalCount __typename} hasViewerReacted {id reacted __typename} nickname showPaymentInfo __typename} starDistribution {score count __typename} hideProductSelectBox total showRecommendationSort itemReviewStats {score count itemId starDistribution {score count __typename} __typename} reactionTypes {name emojiUrl label __typename} __typename}}"
    }
}

r = requests.post(url_l, json=payload, headers= headers)
print(r.text)


{"errors":[{"message":"Internal server error","extensions":{"code":"INTERNAL_SERVER_ERROR"}}]}



In [26]:
import requests

# 요청할 URL 및 헤더 설정
url = "https://pcmap-api.place.naver.com/graphql"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://pcmap.place.naver.com/restaurant/1235520676/review/visitor?reviewMenu=%EB%B0%B1%EA%B9%80%EC%B9%98&reviewSort=recent",
    "Origin": "https://pcmap.place.naver.com"
}

# 요청할 payload 설정
payload = {
    "0": {
        "operationName": "getVisitorReviews",
        "variables": {
            "id": "1235520676",
            "input": {
                "businessId": "1235520676",
                "businessType": "restaurant",
                "item": "0",
                "bookingBusinessId": "920095",
                "page": 5,
                "size": 10,
                "sort": "recent",
                "getReactions": True,
                "getTrailer": True,
                "getUserStats": True,
                "includeContent": True,
                "includeReceiptPhotos": True,
                "isPhotoUsed": False,
                "menu": "백김치",
                "cidList": ["220036", "220039", "220091", "220861"]
            }
        },
        "query": "query getVisitorReviews($input: VisitorReviewsInput) {visitorReviews(input: $input) {items {id rating author {id nickname from imageUrl borderImageUrl objectId url review {totalCount imageCount avgRating __typename} theme {totalCount __typename} isFollowing followerCount followRequested __typename} body thumbnail media {type thumbnail thumbnailRatio class videoId videoOriginSource trailerUrl __typename} tags status visitCount viewCount visited created reply {editUrl body editedBy created date replyTitle isReported isSuspended __typename} originType item {name code options __typename} language highlightOffsets apolloCacheId translatedText businessName showBookingItemName bookingItemName votedKeywords {code iconUrl iconCode displayName __typename} userIdno loginIdno receiptInfoUrl reactionStat {id typeCount {name count __typename} totalCount __typename} hasViewerReacted {id reacted __typename} nickname showPaymentInfo __typename} starDistribution {score count __typename} hideProductSelectBox total showRecommendationSort itemReviewStats {score count itemId starDistribution {score count __typename} __typename} reactionTypes {name emojiUrl label __typename} __typename}}"
    }
}

# POST 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("요청 성공!")
    print(response.text)
else:
    print("요청 실패:", response.status_code)


요청 실패: 500


In [27]:
import requests

url = "https://api.place.naver.com/rest/visitorReview/views"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor",
    "Origin": "https://m.place.naver.com"
}

payload = {
    "businessType": "restaurant",
    "reviewGroupIds": [
        "65aa511d996e8c83087efec9",
        "6576d402ed3f1bdb5ffeff81",
        "65be009e04a2f68ec9341375",
        "65c05b740c50730866492534",
        "65b77ac210993359f0e1a571",
        "65b5d6b2976851dc7ddd418c",
        "656efb450bd5cd1512260c2e",
        "659e0cc421a0752282f34dc9",
        "65a8f9817abea21ccb725beb",
        "656d163200a58e483b73acc7"
    ]
}

response = requests.patch(url, json=payload, headers=headers)

if response.status_code == 204:
    print("Visitor review views updated successfully.")
else:
    print(f"Failed to update visitor review views. Status code: {response.status_code}")


Failed to update visitor review views. Status code: 404


In [9]:
url = "https://cc.naver.com/cc"
payload = {
    "a":"rrv.more&r=&i=&m=0&ssc=Mstore.restaurant&p=a0a1b415646dbe40a2c5a3ed2dc75c90&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=440&py=6314&sx=440&sy=476&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor%23"
    }
response = requests.post(url, data = payload)

In [5]:
response.text

'upstream connect error or disconnect/reset before headers. reset reason: connection termination'

In [19]:
url = "https://nelo2-col.navercorp.com/_store"
payload = {"projectName":"webplayer-coreplayer",
 "projectVersion":"0.7.3-alpha.0",
 "id":"b0b79c85-9a72-4250-8176-765018cdfff0",
 "os":"Windows",
 "osVersion":"10",
 "browser":"Chrome",
 "browserVersion":"121.0.0.0",
 "device":";Chrome/121.0.0.0",
 "type":"init",
 "module":{"src":"/assets/deploy/place.8138.bundle.cfc34896bad501828796.js",
 "data-webpack":"@salt/deploy:chunk-8138",
 "charset":"utf-8"},
 "constructor":{"env":"node","version":"unknown"}}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor",
    "Origin": "https://m.place.naver.com"
}
response = requests.post(url, json = payload, headers=headers)
print(response.text)

{"code":200,"message":"Success"}


In [20]:
url = "https://nelo2-col.navercorp.com/_store"
payload = {"projectName":"webplayer-coreplayer",
           "projectVersion":"0.7.3-alpha.0",
           "id":"b0b79c85-9a72-4250-8176-765018cdfff0",
           "os":"Windows","osVersion":"10",
           "browser":"Chrome",
           "browserVersion":"121.0.0.0",
           "device":";Chrome/121.0.0.0",
           "type":"call","api":"play",
           "hasBeenActive":"true",
           "isActive":"true",
           "trackSrc":"https://a01-g-naver-vod.akamaized.net/pup_review/c/read/v2/VOD_ALPHA/pup_review_2024_02_04_0/6711aa84-c343-11ee-98a8-80615f0bcf7c.mp4?__gda__: 1707335007_cfb2131e2b82017519c45a98f53b2de7",
           "trackLabel":"",
           "trackKind":"main"}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor",
    "Origin": "https://m.place.naver.com"
}
response = requests.post(url, json = payload, headers=headers)
print(response.text)

{"code":200,"message":"Success"}


In [22]:
url = "https://nelo2-col.navercorp.com/_store"

payload = {"projectName":"webplayer-coreplayer",
           "projectVersion":"0.7.3-alpha.0",
           "id":"b0b79c85-9a72-4250-8176-765018cdfff0",
           "os":"Windows",
           "osVersion":"10",
           "browser":"Chrome",
           "browserVersion":"121.0.0.0",
           "device":";Chrome/121.0.0.0",
           "type":"warmup",
           "result":"rejected",
           "protocol":"data:"}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor",
    "Origin": "https://m.place.naver.com"
}
response = requests.post(url, json = payload, headers=headers)
print(response.text)

{"code":200,"message":"Success"}


In [44]:
import json
import requests

url = "https://cc.naver.com/cc"

payload = {
    "a": "rrv.more",
    "r": "",
    "i": "",
    "m": "0",
    "ssc": "Mstore.restaurant",
    "p": "3682adef733654765e5eaefa6485bf82",
    "lcsurl": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "px": "459",
    "py": "11082",
    "sx": "459",
    "sy": "471",
    "ua_brand_0": "Not A(Brand",
    "ua_version_0": "99",
    "ua_brand_1": "Google Chrome",
    "ua_version_1": "121",
    "ua_brand_2": "Chromium",
    "ua_version_2": "121",
    "ua_architecture": "x86",
    "ua_platform": "Windows",
    "ua_platformVersion": "15.0.0",
    "ua_uaFullVersion": "121.0.6167.140",
    "ua_full_brand_0": "Not A(Brand",
    "ua_full_version_0": "99.0.0.0",
    "ua_full_brand_1": "Google Chrome",
    "ua_full_version_1": "121.0.6167.140",
    "ua_full_brand_2": "Chromium",
    "ua_full_version_2": "121.0.6167.140",
    "u": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor%23"
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor",
    "Origin": "https://m.place.naver.com"
}

response = requests.post(url, json=payload, headers=headers)
print(response.text)


In [48]:
import json
import requests

url = "https://api.place.naver.com/graphql"

payload = {
    "operations": "[{operationName: 'getVisitorReviews', variables: {,...},...}]",
    "map": {
        "0": "{\"operationName\":\"getVisitorReviews\",\"variables\":{\"id\":\"1109024709\",\"input\":{\"businessId\":\"1109024709\",\"businessType\":\"restaurant\",\"item\":\"0\",\"bookingBusinessId\":\"496515\",\"page\":5,\"size\":10,\"cidList\":[\"220036\",\"220039\",\"220091\",\"1005863\"],\"getReactions\":true,\"getTrailer\":true,\"getUserStats\":true,\"includeContent\":true,\"includeReceiptPhotos\":true,\"isPhotoUsed\":false}},\"query\":\"query getVisitorReviews($input: VisitorReviewsInput) {...}\""
    }
}

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko",
    "Content-Length": "2590",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZM=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
}

response = requests.post(url, json=payload, headers=headers)
print(response.text)


{"errors":[{"message":"Internal server error","extensions":{"code":"INTERNAL_SERVER_ERROR"}}]}



In [52]:


headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko",
    "Access-Control-Request-Headers": "content-type,x-wtm-graphql",
    "Access-Control-Request-Method": "POST",
    "Content-Length": "2590",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZM=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
}

response = requests.options(url, headers=headers)
print(response.text)

<!DOCTYPE html>
<html lang="ko">
<head>
<meta name="robots" content="noindex, nofollow" />
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<title> :: ìë¹ì¤ì ì ìí  ì ììµëë¤.:: </title>
<style>
body, p, h1, h2, h3, h4, h5, h6, ul, ol, li, dl, dt, dd, table, th, td, form, fieldset, legend, input, textarea, button, select {margin: 0;padding: 0}
body, input, textarea, select, button, table {font-size: 12px;font-family: 'ëëê³ ë', Gulim, helvetica, sans-serif;color: #424242}
body, html {height: 100%}
img, fieldset{border: 0}
img {vertical-align: middle}
ul, ol {list-style: none }
em, address {font-style: normal}
a {color: #000; text-decoration: none}
legend {display: none}
hr { display: none !important }
#u_skip { position: relative; width: 100%; z-index: 10 }
#u_skip a { position: absolute; top: -40px; left: 0; width: auto; padding: 0 10px; border: 1px solid #4ec53d; background-color: #000; color: #fff; line-height: 38px; white-space: nowrap; opaci

In [17]:
import json
import requests

url = "https://api.place.naver.com/graphql"

payload = [{
    "operationName": "getVisitorReviews",
    "variables": {
        "input": {
            "businessId": "1109024709",
            "businessType": "restaurant",
            "item": "0",
            "bookingBusinessId": "496515",
            "page": 1,
            "size": 50,
            "isPhotoUsed": False,
            "includeContent": True,
            "getUserStats": True,
            "includeReceiptPhotos": True,
            "cidList": ["220036", "220039", "220091", "1005863"],
            "getReactions": True,
            "getTrailer": True
        },
        "id": "1109024709"
    },
    "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        borderImageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        isFollowing\n        followerCount\n        followRequested\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        thumbnailRatio\n        class\n        videoId\n        videoOriginSource\n        trailerUrl\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        date\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      bookingItemName\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      loginIdno\n      receiptInfoUrl\n      reactionStat {\n        id\n        typeCount {\n          name\n          count\n          __typename\n        }\n        totalCount\n        __typename\n      }\n      hasViewerReacted {\n        id\n        reacted\n        __typename\n      }\n      nickname\n      showPaymentInfo\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    reactionTypes {\n      name\n      emojiUrl\n      label\n      __typename\n    }\n    __typename\n  }\n}"
}]

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko",
    "Access-Control-Request-Headers": "content-type,x-wtm-graphql",
    "Access-Control-Request-Method": "POST",
    "Content-Length": "2590",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZ=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.json())

[{'data': {'visitorReviews': {'items': [{'id': '65c1a25045c21815624ae762', 'rating': None, 'author': {'id': '3nFy9', 'nickname': 'Minapaku', 'from': 'NAVER', 'imageUrl': 'https://phinf.pstatic.net/contact/20201021_134/16032591306244CwDB_JPEG/Colorado.jpg', 'borderImageUrl': 'https://myplace-phinf.pstatic.net/20240130_261/1706598927925qRvGS_PNG/valentine_border.png', 'objectId': '5da32c858f87a842bcf8a6b6', 'url': 'https://m.place.naver.com/my/5da32c858f87a842bcf8a6b6/review?v=2', 'review': {'totalCount': 1366, 'imageCount': 941, 'avgRating': 4.2, '__typename': 'AuthorReview'}, 'theme': {'totalCount': 0, '__typename': 'AuthorTheme'}, 'isFollowing': None, 'followerCount': '85', 'followRequested': None, '__typename': 'VisitorReviewAuthor'}, 'body': '토마토랑 마라 먹었는데 정말 강력크합니다!ㅋㅋㅋ맵찔이 분들은 토마토 추천드랴요!😙😙 사장님도 넘 친절하시고 맛있어여! 민물 특유의 흙맛이 나긴 하는데 야채추가해서 같이 드시먄 맛있습니다~! 또 오고 시퍼여ㅎㅎㅎ 먹고 냄새 배이는건 각오 하셔야해욬ㅋㅋㅋㅋㅋ온몸에서 마라 냄새 진동', 'thumbnail': 'https://pup-review-phinf.pstatic.net/MjAyNDAyMDZfMjM1/MDAxNzA3MTg4ODE0MT

In [18]:
import pandas as pd
a = response.json()
df = pd.DataFrame(a[0]['data']['visitorReviews']['items'])

In [19]:
df.to_csv('data.csv', index=False)

In [20]:
df['body'].head(3)

0    토마토랑 마라 먹었는데 정말 강력크합니다!ㅋㅋㅋ맵찔이 분들은 토마토 추천드랴요!😙😙...
1    두번째 방문이에요 >_<\n지난번엔 칭화지아오 먹었는데 이번엔 마라🌶🌶🌶\n야채모듬...
2    훠궈를 너무 좋아해서 찾아보다 발견한집\n강남 홍대 지점이\n있어 가까운 강남점 방...
Name: body, dtype: object

In [51]:
import json
import requests

url = "https://api.place.naver.com/graphql"

payload = {
    "operations": "[{operationName: 'getVisitorReviews', variables: {,...},...}]",
    "map": {
        "0": "{\"operationName\":\"getVisitorReviews\",\"variables\":{\"id\":\"1109024709\",\"input\":{\"businessId\":\"1109024709\",\"businessType\":\"restaurant\",\"item\":\"0\",\"bookingBusinessId\":\"496515\",\"page\":5,\"size\":10,\"cidList\":[\"220036\",\"220039\",\"220091\",\"1005863\"],\"getReactions\":true,\"getTrailer\":true,\"getUserStats\":true,\"includeContent\":true,\"includeReceiptPhotos\":true,\"isPhotoUsed\":false}},\"query\":\"query getVisitorReviews($input: VisitorReviewsInput) {...}\""
    }
}

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko",
    "Access-Control-Request-Headers": "content-type,x-wtm-graphql",
    "Access-Control-Request-Method": "POST",
    "Content-Length": "2590",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZM=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
}

response = requests.post(url, json=payload, headers=headers)
print(response.text)


{"errors":[{"message":"Internal server error","extensions":{"code":"INTERNAL_SERVER_ERROR"}}]}



In [42]:
import requests

url = "https://api.place.naver.com/rest/visitorReview/views"


payload = {
    "reviewGroupIds": [
        "65aa511d996e8c83087efec9",
        "6576d402ed3f1bdb5ffeff81",
        "65be009e04a2f68ec9341375",
        "65c05b740c50730866492534",
        "65b77ac210993359f0e1a571",
        "65b5d6b2976851dc7ddd418c",
        "656efb450bd5cd1512260c2e",
        "659e0cc421a0752282f34dc9",
        "65a8f9817abea21ccb725beb",
        "656d163200a58e483b73acc7"
    ],
    "businessType": "restaurant"
}

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZC=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/1109024709/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F1109024709%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

response = requests.patch(url, json=payload, headers=headers)
print(response.text)

In [43]:
response

<Response [204]>

In [41]:
response = requests.patch(url, json=payload, headers=headers)
if response.status_code == 204:
    print("요청이 성공적으로 처리되었습니다.")
    print("응답 헤더:")
    print(response.headers)
else:
    print("요청에 문제가 있습니다. 응답 코드:", response.status_code)


요청이 성공적으로 처리되었습니다.
응답 헤더:
{'access-control-allow-credentials': 'true', 'access-control-allow-origin': 'https://m.place.naver.com', 'date': 'Wed, 07 Feb 2024 11:40:01 GMT', 'set-cookie': 'NID_AUT=expired; expires=Wed, 02-Jun-1999 00:00:00 GMT; path=/; domain=.naver.com;, NID_SES=expired; expires=Wed, 02-Jun-1999 00:00:00 GMT; path=/; domain=.naver.com;', 'vary': 'Accept-Encoding, Origin', 'x-xss-protection': '1; mode=block', 'referrer-policy': 'unsafe-url', 'server': 'nfront'}
